In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

# Generating Data

In [2]:
def generate_date(year_1 = 1980, year_2 = 2000):
    return datetime.datetime(year_1, 1, 1) + datetime.timedelta(days= np.random.randint((datetime.datetime(year_2, 12,31) - datetime.datetime(year_1, 1,1)).days))  

In [3]:
emp_counts = int(1e3)
# Creating DataFrame
data = pd.DataFrame(data = {'id' : np.random.randint(0, emp_counts*10, emp_counts), 
                            'gender' : ['Male' if np.random.rand()<0.5 else 'Female' for i in range(emp_counts)],
                            'dob' : [generate_date() for i in range(emp_counts)],
                            'doh' : [generate_date(year_1 = 2008, year_2 = 2023) for i in range(emp_counts)],
                            'salary' : 6e6+ np.random.uniform(low = -3e6, high = 10e6, size = emp_counts) },)
data['dob'] = pd.to_datetime(data['dob'])
data['doh'] = pd.to_datetime(data['doh'])
data

,id,gender,dob,doh,salary
0,6293,Female,1981-05-20,2018-08-03,1.469778e+07
1,9223,Female,1984-04-04,2012-08-09,1.191939e+07
2,9422,Male,1990-09-08,2013-04-08,8.659652e+06
3,2647,Male,1982-06-06,2013-12-14,3.498368e+06
4,9350,Female,1999-05-25,2022-09-17,1.143665e+07
...,...,...,...,...,...
995,1661,Male,2000-05-20,2008-11-18,1.119378e+07
996,4171,Male,1999-02-16,2011-05-07,1.474486e+07
997,1042,Male,1982-03-15,2009-09-01,5.778569e+06
998,9717,Male,1988-03-26,2014-05-28,8.230075e+06


In [4]:
val_date = pd.Timestamp('2023-12-31')
data['age'] = np.round((val_date- data.dob)/np.timedelta64(1, 'Y'),2)
data['yos'] = np.round((val_date- data.doh)/np.timedelta64(1, 'Y'),2)

# Actuarial Assumptions

# Demographic Assumptions

Assume death probability follows 4th Indonesia Mortality Table. Disability probability is 1% of the former mortality table and resignation rate is 1% decreasing linearly from age 22 to age 56 (pension age).

In [5]:
mortality_base = pd.read_csv(r'data/TMI IV.csv')
pension_age = 56

In [6]:
def resignation_rate(entry_age, start_age = 22, end_age = pension_age, start_rate = 0.01, end_rate = 0):
    return start_rate +(end_rate - start_rate)*(np.arange(entry_age, end_age) - start_age)/(end_age - start_age) 

In [7]:
def demographic_table(table = mortality_base, employee_gender = None, employee_age = None, pension_age = pension_age):
    death = mortality_base[employee_gender].loc[int(employee_age):pension_age-1]
    disable = death*0.01
    resign = resignation_rate(entry_age = employee_age)
    return pd.DataFrame(data = {'death': death, 'disable': disable.values, 'resign' : resign}, 
                        index = np.arange(np.floor(employee_age), pension_age))
    

In [8]:
demographic_table(table = mortality_base,
                  employee_gender = data.gender.iloc[0],
                  employee_age = data.age.iloc[0]).shape

(14, 3)

In [9]:
def service_table(demographic_tbl):
    survive = np.ones((demographic_tbl.shape[0], 4))
    for i in range(demographic_tbl.shape[0]):
        survive[i,1] = survive[i,0] * demographic_tbl.iloc[i,0] * (1 - demographic_tbl.iloc[i,1]) * (1 - demographic_tbl.iloc[i,2])
        survive[i,2] = survive[i,0] * demographic_tbl.iloc[i,1] * (1 - demographic_tbl.iloc[i,0]) * (1 - demographic_tbl.iloc[i,2])
        survive[i,3] = survive[i,0] * demographic_tbl.iloc[i,2] * (1 - demographic_tbl.iloc[i,0]) * (1 - demographic_tbl.iloc[i,1])
        try : 
            survive[i+1,0] = survive[i,0] - np.sum(survive[i,1:])  
        except : 
            survive = np.append(survive, np.array([[survive[i,0] - np.sum(survive[i,1:]), 0, 0, 0]]), axis = 0)
    return pd.DataFrame(data = survive, columns = ['survive', 'death', 'disable', 'resign'], index = np.append(demographic_tbl.index, 56))

In [10]:
service_table(demographic_tbl = demographic_table(table = mortality_base,
                  employee_gender = data.gender.iloc[0],
                  employee_age = data.age.iloc[0]))

,survive,death,disable,resign
42.0,1.000000,0.001404,0.000014,0.003924
43.0,0.994658,0.001526,0.000015,0.003610
44.0,0.989506,0.001667,0.000017,0.003300
45.0,0.984522,0.001835,0.000018,0.002994
46.0,0.979674,0.002042,0.000020,0.002691
47.0,0.974921,0.002237,0.000022,0.002392
48.0,0.970270,0.002449,0.000024,0.002095
49.0,0.965701,0.002670,0.000027,0.001801
50.0,0.961203,0.002927,0.000029,0.001511
51.0,0.956737,0.003201,0.000032,0.001223


## Financial Assumption

In [11]:
sev_svc = pd.DataFrame({'severance': [min(i+1,9) for i in range(60)],
                        'service' : [0,0,0,2,2,2,3,3,3,
                                     4,4,4,5,5,5,6,6,6,
                                     7,7,7,8,8,8,10,10,10,10,10,10,10,
                                     10,10,10,10,10,10,10,10,10,10,10,10,10,10,
                                     10,10,10,10,10,10,10,10,10,10,10,10,10,10,10]})
ben_fac = pd.DataFrame({'retire': 1.75*sev_svc['severance']+sev_svc['service'],
                        'death': 2*sev_svc['severance']+sev_svc['service'],
                        'disable': 2*sev_svc['severance']+sev_svc['service'],
                        'resign': [1]*sev_svc.shape[0]})
ben_fac

,retire,death,disable,resign
0,1.75,2,2,1
1,3.50,4,4,1
2,5.25,6,6,1
3,9.00,10,10,1
4,10.75,12,12,1
5,12.50,14,14,1
6,15.25,17,17,1
7,17.00,19,19,1
8,18.75,21,21,1
9,19.75,22,22,1


### Discount Rate

In [12]:
yield_curve  = pd.read_csv(r'data/YieldCurve1Sep.csv')
yield_curve

,enor Year,Today
0,0.1,0.061580
1,1.0,0.061591
2,2.0,0.061629
3,3.0,0.061763
4,4.0,0.062007
5,5.0,0.062345
6,6.0,0.062754
7,7.0,0.063204
8,8.0,0.063671
9,9.0,0.064134


In [13]:
def spot_rates(yield_curve):
    spot_rate = np.zeros(yield_curve.shape[0])
    t = yield_curve.iloc[:,0]
    spot_rate[0] = yield_curve.iloc[0,1]
    spot_rate[1] = yield_curve.iloc[1,1]
    for i in range(2,yield_curve.shape[0]):
        sum = 0
        for j in range(1,i):
            sum += yield_curve.iloc[j,1]/(1+spot_rate[j])**t[j]
        spot_rate[i] = ((1+yield_curve.iloc[i,1])/(1-sum))**(1/t[i]) - 1   
    return pd.DataFrame(data = {'spot_rate' : spot_rate})

In [23]:
def discount_factor(employee_age, pension_age, rate, type = 'multi-rate'):
    if type == 'multi-rate':
        pass
    elif type == 'single-rate':
        rate = pd.DataFrame([rate]*(pension_age-int(employee_age)))
    else :
        raise 'Please define type of rate that being used either "multi-rate" or "single-rate"'
    return np.append(np.array([(1+rate.iloc[i][0])**-i for i in range(pension_age-int(employee_age))]),
        (1+rate.iloc[pension_age-int(employee_age)-1][0])**-(pension_age-employee_age))

In [24]:
discount_factor(employee_age= data.age.iloc[0], pension_age = pension_age ,rate= spot_rates(yield_curve), type ='multi-rate')

array([1.        , 0.94198236, 0.88729901, 0.83568462, 0.78689182,
       0.74071213, 0.69697421, 0.65554137, 0.61630088, 0.57915721,
       0.54402464, 0.51082366, 0.4794778 , 0.449911  , 0.44006911])

### Salary Increase 

In [20]:
salary_inc = 0.05
def salary_factor(employee_age, pension_age, salary_inc = salary_inc) :
    return np.append(np.array([(1+salary_inc)**i for i in range(pension_age - int(employee_age))]),
    (1+salary_inc)**(pension_age - employee_age))

In [21]:
salary_factor(employee_age= data.age.iloc[0], pension_age = pension_age)

array([1.        , 1.05      , 1.1025    , 1.157625  , 1.21550625,
       1.27628156, 1.34009564, 1.40710042, 1.47745544, 1.55132822,
       1.62889463, 1.71033936, 1.79585633, 1.88564914, 1.91906213])

### Defined Benefit Obligation Factor

In [71]:
def dbo_factor(employee_age, employee_yos, pension_age = pension_age):
    return np.array([employee_yos/(employee_yos+i) for i in range(pension_age-int(employee_age))])

In [72]:
dbo_factor(employee_age=data.age.iloc[0], employee_yos=data.yos.iloc[0])

array([1.        , 0.84399376, 0.73009447, 0.64328181, 0.5749203 ,
       0.5196926 , 0.47414549, 0.43593876, 0.40343028, 0.37543373,
       0.35107073, 0.32967703, 0.31074095, 0.29386203])

# Present Value Benefit

## Combining All Financial and Demographic Assumptions Excluding Pension Calculation

This approach can be rearranged if there is tax gross up. If there is tax gross up, flop will increase at least $n$ times with $2n$ with $n$ is 55-$\lfloor \text{entry age}\rfloor$

In [77]:
financial_assumption = np.multiply(np.multiply(discount_factor(employee_age= data.age.iloc[0], pension_age = pension_age ,rate= spot_rates(yield_curve), type ='multi-rate'),
            salary_factor(employee_age= data.age.iloc[0], pension_age = pension_age),
            ),np.append(dbo_factor(employee_age=data.age.iloc[0], employee_yos=data.yos.iloc[0]),1))
financial_assumption

array([1.        , 0.8347786 , 0.71421284, 0.62231688, 0.54989513,
       0.49129516, 0.44285762, 0.40211538, 0.3673463 , 0.33731328,
       0.31110442, 0.28803282, 0.26757072, 0.24930499, 0.84451996])

In [78]:
svc_table = service_table(demographic_tbl = demographic_table(table = mortality_base,
                  employee_gender = data.gender.iloc[0],
                  employee_age = data.age.iloc[0]))

In [79]:
svc_table_dec = np.multiply(svc_table.iloc[:-1,1:].values,np.vstack((np.vstack((financial_assumption[:-1],financial_assumption[:-1])),financial_assumption[:-1])).T)
svc_table_dec

array([[1.40443973e-03, 1.40247924e-05, 3.92381597e-03],
       [1.27402319e-03, 1.27208078e-05, 3.01375886e-03],
       [1.19034263e-03, 1.18835104e-05, 2.35723893e-03],
       [1.14220821e-03, 1.14009360e-05, 1.86336081e-03],
       [1.12279785e-03, 1.12047462e-05, 1.47992882e-03],
       [1.09890591e-03, 1.09640364e-05, 1.17497648e-03],
       [1.08473884e-03, 1.08202182e-05, 9.27779020e-04],
       [1.07361385e-03, 1.07066960e-05, 7.24360928e-04],
       [1.07520772e-03, 1.07196103e-05, 5.54928431e-04],
       [1.07968938e-03, 1.07610847e-05, 4.12440724e-04],
       [1.08911520e-03, 1.08514719e-05, 2.91685343e-04],
       [1.09938115e-03, 1.09499477e-05, 1.88722699e-04],
       [1.11510748e-03, 1.11022778e-05, 1.00508865e-04],
       [1.13017070e-03, 1.12476630e-05, 2.46583356e-05]])

In [81]:
dbo_exclude_pension = svc_table_dec*data.salary.iloc[0]
dbo_exclude_pension

array([[20642.14255727,   206.13327784, 57671.37388679],
       [18725.30927605,   186.9676023 , 44295.55710029],
       [17495.3911437 ,   174.6611911 , 34646.17315799],
       [16787.92201933,   167.56841958, 27387.26254547],
       [16502.63286962,   164.68486558, 21751.66441236],
       [16151.47446645,   161.14696713, 17269.54280788],
       [15943.25004354,   159.03315975, 13636.28956881],
       [15779.73743258,   157.3646346 , 10646.49572365],
       [15803.16373006,   157.55444622,  8156.21458713],
       [15869.03418719,   158.16402772,  6061.96198064],
       [16007.5728512 ,   159.49251916,  4287.12626295],
       [16158.45949406,   160.9398949 ,  2773.80423491],
       [16389.60159057,   163.17880802,  1477.25692553],
       [16610.99741594,   165.31564773,   362.42272854]])